### Data Import

In [17]:
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import XLMRobertaTokenizer
import pyLDAvis.gensim_models as gensimvis
import torch
from tqdm import tqdm
import torch.nn.functional as F
from scipy.special import softmax
import re



### RoBERTa pre-trained on Twitter

As a first try, we want to use a pre-trained model called XLM-Roberta-base that is trained on ~198M multilingual tweets (among which Italian) and we implement the version for sentiment analysis.

Source: https://arxiv.org/abs/2104.12250
https://huggingface.co/cardiffnlp/twitter-xlm-roberta-base-sentiment

In [ ]:
MODEL = "cardiffnlp/twitter-xlm-roberta-base-sentiment"

tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.eval()
# Define labels
labels = ['negative', 'neutral', 'positive']

In [5]:
politicians_cleaned = pd.read_csv('../politicians_data/politicians_classified.csv')

In [14]:
# Twitter-specific preprocessing
def clean_tweet(text):
    text = str(text)
    text = re.sub(r"http\S+|www.\S+", "", text)      # Remove URLs
    text = re.sub(r"@\w+", "@user", text)            # Replace mentions
    text = re.sub(r"#", "", text)                    # Remove hashtag symbols
    text = re.sub(r"\s+", " ", text).strip()         # Remove extra spaces
    return text.lower()

In [19]:
# Sentiment analysis function (CPU version)
def get_sentiment(text):
    text = clean_tweet(text)
    encoded_input = tokenizer(text, return_tensors='pt', truncation=True, max_length=128)
    with torch.no_grad():
        output = model(**encoded_input)
        scores = F.softmax(output.logits, dim=1)[0]

    label = labels[torch.argmax(scores).item()]
    return {
        'label': label,
        'scores': {labels[i]: float(scores[i]) for i in range(len(labels))}
    }




In [23]:
# Apply sentiment analysis with progress bar
tqdm.pandas(desc="Analyzing sentiment")
roberta_result = politicians_cleaned['Content'].progress_apply(get_sentiment)


Analyzing sentiment: 100%|██████████| 17245/17245 [12:02<00:00, 23.88it/s]


In [34]:
roberta_result= roberta_result.to_dict()

In [35]:
roberta_result_df = pd.DataFrame.from_dict(roberta_result, orient='index', columns=['label'])
roberta_result_df

,label
0,negative
1,negative
2,negative
3,negative
4,negative
...,...
17240,negative
17241,positive
17242,positive
17243,positive


In [36]:
roberta_sentiment = politicians_cleaned[['ID', 'Content', 'politician', 'party']].copy()
roberta_sentiment['label'] = roberta_result_df['label']

In [38]:
roberta_sentiment['label'].value_counts()

label
positive    10333
negative     5144
neutral      1768
Name: count, dtype: int64

In [39]:
# print a sample of 5 'Content' per each sentiment label
for label in roberta_sentiment['label'].unique():
    print(f"Sample {label} tweets:")
    samples = roberta_sentiment[roberta_sentiment['label'] == label]['Content'].sample(5, random_state=42).to_list()
    for n,s in enumerate(samples):
        print(f"Tweet {n+1}: {s}\n")
    print("\n")

Sample negative tweets:
Tweet 1: Vaccinarsi è una scelta che protegge noi stessi e le persone a cui vogliamo bene. Per questo ben venga il #greenpass.
Ma il governo faccia anche di più: più tracciamento, più mezzi pubblici in sicurezza, meno alunni per classe https://t.co/ABQ2e7jygt

Tweet 2: Il voto utile non esiste. Ci sono 4 coalizioni e la vittoria di questa destra poco seria e sfascista si può fermare eccome. L’unica vera rivoluzione che serve a questo paese è quella della serietà e del buon governo.

A #Brescia per la presentazione dei nostri candidati. https://t.co/yBjsBMWhj0

Tweet 3: Grazie a tutti coloro che in questa calda domenica sono venuti e stanno venendo a firmare i #referendumgiustizia, un’occasione di democrazia e libertà che va al di là degli schieramenti politici. Si può firmare tutta estate in tutti i Municipi italiani 🇮🇹 Un saluto da Fregene! https://t.co/U7JKEN4fsh

Tweet 4: Piena solidarietà a @sbonaccini. Vai avanti Stefano! Siamo con te.

Tweet 5: Siamo il ce

We are not much happy with the results because, over these 15 instances, it correctly classified at most 4 tweets. So we will try to implement another model.

## Model trained only on italian tweets

As a second shot, we apply a pre-trained model based on UmBERTo which has been fine-tuned precisely on Italian tweets and again there is one version made for sentiment analysis.

In [ ]:
model_name = "MilaNLProc/feel-it-italian-emotion"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def italian_sentiment(text):
    text = clean_tweet(text)
    encoded_input = tokenizer(text, return_tensors='pt', truncation=True)
    with torch.no_grad():
        output = model(**encoded_input)
        scores = F.softmax(output.logits, dim=1)[0]
    labels = ['fear', 'joy', 'sadness', 'anger']
    label = labels[torch.argmax(scores).item()]
    return {
        'label': label,
        'scores': {labels[i]: float(scores[i]) for i in range(len(labels))}
    }



In [ ]:
feelit_result = politicians_cleaned['Content'].apply(italian_sentiment)

In [30]:
feelit_result = feelit_result.to_dict()

In [31]:
feelit_df = pd.DataFrame.from_dict(feelit_result, orient='index')
feelit_df

,label,scores
0,negative,"{'positive': 0.000249743927270174, 'negative':..."
1,negative,"{'positive': 0.001125097624026239, 'negative':..."
2,negative,"{'positive': 0.0031859937589615583, 'negative'..."
3,positive,"{'positive': 0.9997900128364563, 'negative': 0..."
4,positive,"{'positive': 0.9927362203598022, 'negative': 0..."
...,...,...
17240,positive,"{'positive': 0.9997599720954895, 'negative': 0..."
17241,positive,"{'positive': 0.9997979998588562, 'negative': 0..."
17242,positive,"{'positive': 0.9997934699058533, 'negative': 0..."
17243,positive,"{'positive': 0.9997965693473816, 'negative': 0..."


In [32]:
feelit_sentiment = politicians_cleaned[['ID', 'Content', 'politician', 'party']].copy()
feelit_sentiment['label'] = feelit_df['label']

In [33]:
feelit_sentiment['label'].value_counts()

label
positive    14035
negative     3210
Name: count, dtype: int64

In [35]:
# print a sample of 5 'Content' per each sentiment label
for label in feelit_sentiment['label'].unique():
    print(f"Sample {label} tweets:")
    samples = feelit_sentiment[feelit_sentiment['label'] == label]['Content'].sample(5, random_state=42).to_list()
    for n,s in enumerate(samples):
        print(f"Tweet {n+1}: {s}\n")
    print("\n")

Sample negative tweets:
Tweet 1: Oggi, con la fine dello stato d'emergenza, si conclude l'incarico del Generale Figliuolo. Il suo lavoro alla guida della campagna vaccinale ha rappresentato un punto di svolta nella lotta al Covid. Era il cambio di passo che @forza_italia voleva al governo. Lo ringraziamo!

Tweet 2: @R__Pizzi @CalendaSindaco 700 per la precisione.

Tweet 3: Andiamo avanti. 
Per un paese più giusto, più verde e più libero.

#verdiesinistra #sinistraitaliana #giustizia https://t.co/aJ0exX0Uqa

Tweet 4: Caso #Open. Per la quinta volta la Cassazione critica i PM fiorentini. Niente risarcisce il dolore di questi mesi ma è un giorno di speranza: quando parla la giustizia, tace il giustizialismo
https://t.co/z78DwFACC5 https://t.co/VrOM2YbbrE

Tweet 5: Buon lavoro al Presidente rieletto Rebelo de Sousa e complimenti per lo straordinario risultato ad André Ventura di Chega 🇵🇹, alleato europeo della Lega, al suo esordio.
P.s. Se in #Portogallo il Covid non ferma la Democrazia, p